# Preprocessing pipeline 2 - bond graph features models

This notebook starts from the output at step 10 (10_network_featureEng.ipynb) in order to prepare the data to be put into the new bond graph featured models for predictions

In [1]:
import numpy as np
import pandas as pd
import pickle
import datetime#
from scripts_ml.preprocessing_pipeline import *

## Importing data and cleaning unnnecessary instruments

In [2]:
datafolder = "../data/"
inputfilename = '04_instrumentsdf_bondgraph.pkl'

#feature selection
feat_str = ['currency']
feat_quant = ['has_purchase', 'dd_value_date', 'cd_lent_c', 'cd_repaid_c', 'cd_impaired1_c', 'cd_pastdue90_c', 'cd_trend_a', 'c_lent_c', 'c_repaid_c', 'c_impaired1_c', 
              'c_pastdue90_c', 'c_trend_a', 'd_repaid_c', 'd_impaired1_c', 'd_pastdue90_c', 'd_trend_a', 'd_we_payment_share', 'flow_shock_imp1', 'imp_c_node_eff', 'imp_energy', 'imp_d_node_flow', 
              'flow_shock_p90', 'p90_c_node_eff', 'p90_energy', 'p90_d_node_flow', 'flow_shock_p180', 'p180_d_node_flow', 'p180_energy']
feat_exp = ['invoice_amount', 'purchase_amount']
feat_date = ['invoice_date']

In [3]:
df = pd.read_pickle(datafolder+inputfilename)
df.head()

,customer_id,customer_name_1,debtor_id,debtor_name_1,invoice_number,invoice_date,due_date,invoice_amount,purchase_amount,purchase_amount_open,...,p90_c_node_eff,p90_node_flow,p90_energy,flow_shock_p90,p180_edge_flow,p180_d_node_flow,p180_c_node_eff,p180_node_flow,p180_energy,flow_shock_p180
uid,,,,,,,,,,,,,,,,,,,,,
2744:79/231,2004008,Castillo GmbH,79,Sana Hyannis Sarl,2744,2013-07-23,2013-08-02,913.70,0.0,0.0,...,239912.0,0.000000,0.000000,1.0,0.0,0.0,239912.0,0.00000,0.00000,1.0
2861:79/232,2004008,Castillo GmbH,79,Sana Hyannis Sarl,2861,2013-07-30,2013-08-09,2233.45,0.0,0.0,...,239912.0,0.000000,0.000000,1.0,0.0,0.0,239912.0,0.00000,0.00000,1.0
2932:79/233,2004008,Castillo GmbH,79,Sana Hyannis Sarl,2932,2013-08-06,2013-08-16,1370.50,0.0,0.0,...,239912.0,0.000000,0.000000,1.0,0.0,0.0,239912.0,0.00000,0.00000,1.0
1472:489/688,2004009,Orpheus Wyandotte Supply LLC,489,Isfahan SA,1472,2013-08-13,2013-08-23,9195.10,0.0,0.0,...,234247.0,1.665866,366437.206374,36.0,0.0,0.0,234247.0,0.54333,119515.16508,1.0
2042:512/645,2004009,Orpheus Wyandotte Supply LLC,512,Aldrich Chloe GmbH,2042,2013-08-13,2013-08-23,4594.60,0.0,0.0,...,234247.0,1.665866,366437.206374,36.0,0.0,0.0,234247.0,0.54333,119515.16508,1.0


In [4]:
#drop all instruments that are not due yet, since they can't be labelled
print("{:} instruments that are not due yet, dropping...".format(sum(~df.is_due)))
df=df.loc[df.is_due, :]
print("{:} instruments remaining".format(df.shape[0]))

2201 instruments that are not due yet, dropping...
57619 instruments remaining


## Pipeline

In [5]:
targets = ['has_impairment1', 'is_pastdue90', 'is_pastdue180']

### Generating preprocessed data for impairment, pastdue90 and pastdue180 credit events - Shuffle mode

In [6]:
pfixes = ['imp_bg_', 'p90_bg_', 'p180_bg_']
output_path = datafolder+"/preproc_traintest/"

In [9]:
for t in range(len(targets)):
    y_train, X_train, y_test, X_test, feature_labels = preprocessing_pipeline(df, feat_str, feat_quant, feat_exp, feat_date, targets[t], 'invoice_date', 
                                                                              'enriched_shuffle', trainsize = int(df.shape[0]*.80), testsize = int(df.shape[0]*.20)-1,
                                                                         save_to_file=True, outputpath=output_path, prefix=pfixes[t])

Sampling 46095 for train and 11522 for test sets by shuffling...
Running the pipeline, target feature is has_impairment1...
Train y: 46095 total, 968 class_1 observations (2.10%) > 0
pipeline fit_transform for train set...
Test y: 11522 total, 227 class_1 observations (1.97%) > 0
pipeline transform only for test set...
Saving with file name prefix shuffle_imp_bg_ and postfix _190721_1655...
...done.
Sampling 46095 for train and 11522 for test sets by shuffling...
Running the pipeline, target feature is is_pastdue90...
Train y: 46095 total, 3344 class_1 observations (7.25%) > 0
pipeline fit_transform for train set...
Test y: 11522 total, 850 class_1 observations (7.38%) > 0
pipeline transform only for test set...
Saving with file name prefix shuffle_p90_bg_ and postfix _190721_1655...
...done.
Sampling 46095 for train and 11522 for test sets by shuffling...
Running the pipeline, target feature is is_pastdue180...
Train y: 46095 total, 2865 class_1 observations (6.22%) > 0
pipeline fit_t

### Generating preprocessed data for impairment, pastdue90 and pastdue180 credit events - Timewise mode

In [12]:
for t in range(len(targets)):
    if t!=2:
        tdate = datetime.datetime(2018, 4, 30)
        y_train, X_train, y_test, X_test, feature_labels = preprocessing_pipeline(df, feat_str, feat_quant, feat_exp, feat_date, targets[t],
                                                                             'invoice_date', 'enriched_time', timewise=True, testdate = tdate,
                                                                             save_to_file=True, outputpath=output_path, prefix=pfixes[t])
    else:
        tdate = datetime.datetime(2018, 2, 20)
        y_train, X_train, y_test, X_test, feature_labels = preprocessing_pipeline(df, feat_str, feat_quant, feat_exp, feat_date, targets[t],
                                                                             'invoice_date', 'enriched_time', timewise=True, testdate = tdate,
                                                                             save_to_file=True, outputpath=output_path, prefix=pfixes[t])

Splitting train and test sets by time, test cutoff: 2018-04-30 00:00:00...
  46101(80.0%) train, 11518(20.0%) test
Running the pipeline, target feature is has_impairment1...
Train y: 46101 total, 710 class_1 observations (1.54%) > 0
pipeline fit_transform for train set...
Test y: 11518 total, 485 class_1 observations (4.21%) > 0
pipeline transform only for test set...
Saving with file name prefix time_2018-04-30_imp_bg_ and postfix _190721_170...
...done.
Splitting train and test sets by time, test cutoff: 2018-04-30 00:00:00...
  46101(80.0%) train, 11518(20.0%) test
Running the pipeline, target feature is is_pastdue90...
Train y: 46101 total, 3973 class_1 observations (8.62%) > 0
pipeline fit_transform for train set...
Test y: 11518 total, 222 class_1 observations (1.93%) > 0
pipeline transform only for test set...
Saving with file name prefix time_2018-04-30_p90_bg_ and postfix _190721_170...
...done.
Splitting train and test sets by time, test cutoff: 2018-02-20 00:00:00...
  42079

### Generating preprocessed data for impairment, pastdue90 and pastdue180 credit events - Sequential Timewise mode

In [7]:
inputfilename = '03_instrumentsdf_deg1stats.pkl'
df = pd.read_pickle(datafolder+inputfilename)
df.head()

,customer_id,customer_name_1,debtor_id,debtor_name_1,invoice_number,invoice_date,due_date,invoice_amount,purchase_amount,purchase_amount_open,...,c_pastdue90_c,c_pastdue180_c,c_trend_a,c_we_payment_share,c_pd_mismatch_mean,c_pd_mismatch_std,c_repaid_r,c_impaired1_r,c_pastdue90_r,c_pastdue180_r
uid,,,,,,,,,,,,,,,,,,,,,
2744:79/231,2004008,Castillo GmbH,79,Sana Hyannis Sarl,2744,2013-07-23,2013-08-02,913.70,0.0,0.0,...,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2861:79/232,2004008,Castillo GmbH,79,Sana Hyannis Sarl,2861,2013-07-30,2013-08-09,2233.45,0.0,0.0,...,0.0,0.0,0.000000,NaN,NaN,NaN,0.0,0.0,0.0,0.0
2932:79/233,2004008,Castillo GmbH,79,Sana Hyannis Sarl,2932,2013-08-06,2013-08-16,1370.50,0.0,0.0,...,0.0,0.0,7.185198,NaN,NaN,NaN,0.0,0.0,0.0,0.0
1472:489/688,2004009,Orpheus Wyandotte Supply LLC,489,Isfahan SA,1472,2013-08-13,2013-08-23,9195.10,0.0,0.0,...,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2042:512/645,2004009,Orpheus Wyandotte Supply LLC,512,Aldrich Chloe GmbH,2042,2013-08-13,2013-08-23,4594.60,0.0,0.0,...,0.0,0.0,0.000000,NaN,NaN,NaN,0.0,0.0,0.0,0.0


In [8]:
bond_graph_settings = [
    {'col_to_calc_effort' : 'purchase_amount',
    'effort_col' : 'imp_edge_eff', 
    'flow_col' : 'imp_edge_flow', 
    'col_to_calc_flow' : 'total_impairment',
     'col_ratio_flow' : 'invoice_amount',
    'node_flow_col' : 'imp_node_flow', 
    'energy_col' : 'imp_energy', 
    'c_node_eff_col' : 'imp_c_node_eff',
    'd_node_flow_col' : 'imp_d_node_flow', 
    'shock_col' : 'flow_shock_imp1',
    'red_coeff' : 10**4},
    
    {'col_to_calc_effort' : 'payment_date_mismatch',
    'effort_col' : 'p90_edge_eff', 
    'flow_col' : 'p90_edge_flow', 
    'col_to_calc_flow' : 'cd_pastdue90_r',
     'col_ratio_flow' : 'invoice_amount',
    'node_flow_col' : 'p90_node_flow', 
    'energy_col' : 'p90_energy', 
    'c_node_eff_col' : 'p90_c_node_eff',
    'd_node_flow_col' : 'p90_d_node_flow', 
    'shock_col' : 'flow_shock_p90',
    'red_coeff' : 10**4},
    
    {'col_to_calc_effort' : 'payment_date_mismatch',
    'effort_col' : 'p180_edge_eff', 
    'flow_col' : 'p180_edge_flow', 
    'col_to_calc_flow' : 'cd_pastdue180_r',
     'col_ratio_flow' : 'invoice_amount',
    'node_flow_col' : 'p180_node_flow', 
    'energy_col' : 'p180_energy', 
    'c_node_eff_col' : 'p180_c_node_eff',
    'd_node_flow_col' : 'p180_d_node_flow', 
    'shock_col' : 'flow_shock_p180',
    'red_coeff' : 10**4}
]

for t in range(len(targets)):
    if t!=2:
        tdate = datetime.datetime(2018, 4, 30)
        y_train, X_train, y_test, X_test, feature_labels = preproc_pipeline_timeseq(df, feat_str, feat_quant, feat_exp, feat_date, targets[t],
                                                                             'invoice_date', 'enriched_time_seq', bond_graph_settings, testdate = tdate,
                                                                            train_window=12000, test_window=3000,
                                                                             save_to_file=True, outputpath=output_path, prefix=pfixes[t])
    else:
        tdate = datetime.datetime(2018, 2, 20)
        y_train, X_train, y_test, X_test, feature_labels = preproc_pipeline_timeseq(df, feat_str, feat_quant, feat_exp, feat_date, targets[t],
                                                                             'invoice_date', 'enriched_time_seq', bond_graph_settings, testdate = tdate,
                                                                             train_window=12000, test_window=3000,
                                                                                    save_to_file=True, outputpath=output_path, prefix=pfixes[t])

---------Macro train split-----------
Dropping 2201 instruments that are not due...
Splitting train and test sets by time, test cutoff: 2018-04-30 00:00:00...
  46101(80.0%) train, 11518(20.0%) test
---------Adding bond graph features 1 of 3-----------
Calculating effort and flow for starting dataset with shape (46101, 110)...
Starting bg features - dataset shape: (46101, 112)
Creating the undirected graph of the whole dataset network...
Adding effort and flow feature to the dataset...
Dataset shape after effort and flow features: (46101, 116)
Isolating components and creating directed graphs...
Looking for hybrid nodes...
Modelling the flow...
Creating flow graphs...
1477 graphs successfully created
Total calculated shock flow is 58053263 over 136 graphs with positive flow value
Overlapping the flows...
Adding shock-propagation features...
Final dataset shape: (46101, 117)
Done!
---------Adding bond graph features 2 of 3-----------
Calculating effort and flow for starting dataset with

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [9]:
#trainx = []
#testx = []
#for count, train, test, boh in rolling_window(df.shape[0], 12000, 3000):
#    if count==5:
#        trainx=train
#        testx=test

Preparing fold 0 with start at 5820, 14820 train observations and 3000 test observations...
Preparing fold 1 with start at 8820, 12000 train observations and 3000 test observations...
Preparing fold 2 with start at 11820, 12000 train observations and 3000 test observations...
Preparing fold 3 with start at 14820, 12000 train observations and 3000 test observations...
Preparing fold 4 with start at 17820, 12000 train observations and 3000 test observations...
Preparing fold 5 with start at 20820, 12000 train observations and 3000 test observations...
Preparing fold 6 with start at 23820, 12000 train observations and 3000 test observations...
Preparing fold 7 with start at 26820, 12000 train observations and 3000 test observations...
Preparing fold 8 with start at 29820, 12000 train observations and 3000 test observations...
Preparing fold 9 with start at 32820, 12000 train observations and 3000 test observations...
Preparing fold 10 with start at 35820, 12000 train observations and 3000

In [12]:
df.iloc[testx].shape

(3000, 110)